In [51]:

import logging
from getpass import getpass
import asyncio
import json
import slixmpp
from slixmpp.exceptions import IqError, IqTimeout

In [52]:
f = open("topologia.txt", "r")
data = json.load(f)

In [53]:
print(data)

{'type': 'topo', 'config': {'jjhh@alumchat.fun': ['jjhh1@alumchat.fun', 'jjhh3@alumchat.fun', 'jjhh5@alumchat.fun'], 'jjhh1@alumchat.fun': ['jjhh@alumchat.fun', 'jjhh2@alumchat.fun', 'jjhh3@alumchat.fun', 'jjhh5@alumchat.fun'], 'jjhh2@alumchat.fun': ['jjhh1@alumchat.fun', 'jjhh4@alumchat.fun', 'jjhh5@alumchat.fun'], 'jjhh3@alumchat.fun': ['jjhh@alumchat.fun', 'jjhh1@alumchat.fun'], 'jjhh4@alumchat.fun': ['jjhh2@alumchat.fun', 'jjhh5@alumchat.fun'], 'jjhh5@alumchat.fun': ['jjhh@alumchat.fun', 'jjhh1@alumchat.fun', 'jjhh2@alumchat.fun', 'jjhh4@alumchat.fun']}}


In [54]:
data['config']

{'jjhh@alumchat.fun': ['jjhh1@alumchat.fun',
  'jjhh3@alumchat.fun',
  'jjhh5@alumchat.fun'],
 'jjhh1@alumchat.fun': ['jjhh@alumchat.fun',
  'jjhh2@alumchat.fun',
  'jjhh3@alumchat.fun',
  'jjhh5@alumchat.fun'],
 'jjhh2@alumchat.fun': ['jjhh1@alumchat.fun',
  'jjhh4@alumchat.fun',
  'jjhh5@alumchat.fun'],
 'jjhh3@alumchat.fun': ['jjhh@alumchat.fun', 'jjhh1@alumchat.fun'],
 'jjhh4@alumchat.fun': ['jjhh2@alumchat.fun', 'jjhh5@alumchat.fun'],
 'jjhh5@alumchat.fun': ['jjhh@alumchat.fun',
  'jjhh1@alumchat.fun',
  'jjhh2@alumchat.fun',
  'jjhh4@alumchat.fun']}

In [55]:
list(data['config'].keys())


['jjhh@alumchat.fun',
 'jjhh1@alumchat.fun',
 'jjhh2@alumchat.fun',
 'jjhh3@alumchat.fun',
 'jjhh4@alumchat.fun',
 'jjhh5@alumchat.fun']

In [56]:
input_usuario = 'jjhh@alumchat.fun'
user_connections = data['config'][input_usuario]
network_users = list(data['config'].keys())

In [91]:
user_connections

['jjhh1@alumchat.fun', 'jjhh3@alumchat.fun', 'jjhh5@alumchat.fun']

In [57]:
# substract network users from user connections
list(set(network_users) - set(user_connections)- set([input_usuario]))

['jjhh2@alumchat.fun', 'jjhh4@alumchat.fun']

In [58]:


def distance_vector_initialization(input_usuario, user_connections, network_users):
    hops = {}
    distance_vector = {}
    for n in network_users:
        if n in user_connections:
            distance_vector[n] = 1
            hops[n] = input_usuario +","+n
        elif n == input_usuario:
            distance_vector[n] = 0
            hops[n] = input_usuario
        else:
            distance_vector[n] = 1000
            hops[n] = input_usuario
    return distance_vector, hops
    

In [88]:
origin, origin_hops = distance_vector_initialization(input_usuario, user_connections, network_users)
origin, origin_hops

({'jjhh@alumchat.fun': 0,
  'jjhh1@alumchat.fun': 1,
  'jjhh2@alumchat.fun': 1000,
  'jjhh3@alumchat.fun': 1,
  'jjhh4@alumchat.fun': 1000,
  'jjhh5@alumchat.fun': 1},
 {'jjhh@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh1@alumchat.fun': 'jjhh@alumchat.fun,jjhh1@alumchat.fun',
  'jjhh2@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh3@alumchat.fun': 'jjhh@alumchat.fun,jjhh3@alumchat.fun',
  'jjhh4@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh5@alumchat.fun': 'jjhh@alumchat.fun,jjhh5@alumchat.fun'})

In [60]:
first_contact, first_contact_hops = distance_vector_initialization('jjhh1@alumchat.fun', data['config']['jjhh1@alumchat.fun'], network_users)
second_contact, second_contact_hops = distance_vector_initialization('jjhh1@alumchat.fun', data['config']['jjhh3@alumchat.fun'], network_users)

In [61]:
(first_contact, first_contact_hops)

({'jjhh@alumchat.fun': 1,
  'jjhh1@alumchat.fun': 0,
  'jjhh2@alumchat.fun': 1,
  'jjhh3@alumchat.fun': 1,
  'jjhh4@alumchat.fun': 1000,
  'jjhh5@alumchat.fun': 1},
 {'jjhh@alumchat.fun': 'jjhh1@alumchat.fun,jjhh@alumchat.fun',
  'jjhh1@alumchat.fun': 'jjhh1@alumchat.fun',
  'jjhh2@alumchat.fun': 'jjhh1@alumchat.fun,jjhh2@alumchat.fun',
  'jjhh3@alumchat.fun': 'jjhh1@alumchat.fun,jjhh3@alumchat.fun',
  'jjhh4@alumchat.fun': 'jjhh1@alumchat.fun',
  'jjhh5@alumchat.fun': 'jjhh1@alumchat.fun,jjhh5@alumchat.fun'})

In [62]:
# Esto cada vez que le caiga un mensaje
# se me ocurria un split tipo 'computation||dv||hops
def distance_vector_update(self_node, distance_vector, hops_vector, upcomming_distance_vector, upcomming_hops_vector):
    node_list = list(distance_vector.keys())
    result_distance = {}
    result_hops = {}
    for n in node_list:
        # el +1 es porque la distancia hacia mis vecinos se que es 1
        if (upcomming_distance_vector[n] + 1) < distance_vector[n]:
            result_distance[n] = upcomming_distance_vector[n] + 1
            result_hops[n] = self_node +","+upcomming_hops_vector[n]  #la , es para separar los hops que se deben tener
        else:
            #si no se cambia nada se quedan como estaban
            result_distance[n] = distance_vector[n]
            result_hops[n] = hops_vector[n]
    return result_distance, result_hops

simulación de uso con 1 vecino

In [63]:
first_iter_dv, first_iter_hops = distance_vector_update(input_usuario, origin, origin_hops, first_contact, first_contact_hops)

In [64]:
(first_iter_dv, first_iter_hops)

({'jjhh@alumchat.fun': 0,
  'jjhh1@alumchat.fun': 1,
  'jjhh2@alumchat.fun': 2,
  'jjhh3@alumchat.fun': 1,
  'jjhh4@alumchat.fun': 1000,
  'jjhh5@alumchat.fun': 1},
 {'jjhh@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh1@alumchat.fun': 'jjhh@alumchat.fun,jjhh1@alumchat.fun',
  'jjhh2@alumchat.fun': 'jjhh@alumchat.fun,jjhh1@alumchat.fun,jjhh2@alumchat.fun',
  'jjhh3@alumchat.fun': 'jjhh@alumchat.fun,jjhh3@alumchat.fun',
  'jjhh4@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh5@alumchat.fun': 'jjhh@alumchat.fun,jjhh5@alumchat.fun'})

In [90]:
strprueba = 'computation||'+json.dumps(first_iter_dv)+'||'+json.dumps(first_iter_hops)
print(strprueba)

computation||{"jjhh@alumchat.fun": 0, "jjhh1@alumchat.fun": 1, "jjhh2@alumchat.fun": 2, "jjhh3@alumchat.fun": 1, "jjhh4@alumchat.fun": 1000, "jjhh5@alumchat.fun": 1}||{"jjhh@alumchat.fun": "jjhh@alumchat.fun", "jjhh1@alumchat.fun": "jjhh@alumchat.fun,jjhh1@alumchat.fun", "jjhh2@alumchat.fun": "jjhh@alumchat.fun,jjhh1@alumchat.fun,jjhh2@alumchat.fun", "jjhh3@alumchat.fun": "jjhh@alumchat.fun,jjhh3@alumchat.fun", "jjhh4@alumchat.fun": "jjhh@alumchat.fun", "jjhh5@alumchat.fun": "jjhh@alumchat.fun,jjhh5@alumchat.fun"}


In [92]:
json.loads(strprueba.split('||')[1])['jjhh@alumchat.fun']

0

En teoría si le paso el mismo dv y dh no debería de cambiar el resultado

In [65]:
distance_vector_update(input_usuario, first_iter_dv, first_iter_hops, first_contact, first_contact_hops)

({'jjhh@alumchat.fun': 0,
  'jjhh1@alumchat.fun': 1,
  'jjhh2@alumchat.fun': 2,
  'jjhh3@alumchat.fun': 1,
  'jjhh4@alumchat.fun': 1000,
  'jjhh5@alumchat.fun': 1},
 {'jjhh@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh1@alumchat.fun': 'jjhh@alumchat.fun,jjhh1@alumchat.fun',
  'jjhh2@alumchat.fun': 'jjhh@alumchat.fun,jjhh1@alumchat.fun,jjhh2@alumchat.fun',
  'jjhh3@alumchat.fun': 'jjhh@alumchat.fun,jjhh3@alumchat.fun',
  'jjhh4@alumchat.fun': 'jjhh@alumchat.fun',
  'jjhh5@alumchat.fun': 'jjhh@alumchat.fun,jjhh5@alumchat.fun'})

En resumen, cada nodo puede recibir un distance vector y distance hops de un vecino sin necesariamente saber de quién vino: el protocolo que puede funcionar para el mensaje podría ser: 'computation||'+Distance_vector+'||'+Hops_vector

Aqui son pruebas para ver lo del distance vector

La forma de parar estas iteraciones, es ver si despues varios computations que me llegan, mi origin (distance vector) es igual, entonces podemos sacar los values de origin y nos va a devolver una lista de numeros.

Tambien dentro de nuestras clases vamos a tener que crear un atributo lista, que sea como historial_origin

In [5]:
list_1 = [1,2,3,4,5]
list_2 = [1,2,3,4,5]

In [6]:
list_1[1:]

[2, 3, 4, 5]

In [117]:
list_1 == list_2

True

In [75]:
list_of_lists = [list_1, list_2] #las listas de los valores de distancia de las tablas de ruteo

#check if all the list inside list_of_lists are equal
def check_lists(list_of_lists):
    for l in list_of_lists:
        if l != list_of_lists[0]:
            return False
    return True

In [ ]:
#suponiendo de que new_origin fuera un diccionario con los jids y los pesos (el int del costo)
#self.historial_origin = []
#self.historial_origin = (self.historial_origin.copy() + [list(new_origin.values())])[-5:] #esto va a ser una lista de los ultimos 5 vectores


# el boolean para saber si paro de actualizar mi origin y origin_hops seria
# me va a dar verdadero si debo parar
# len(self.historial_origin) == 5 && check_lists(self.historial_origin)


#hacer el update basado en un if de una variable booleana que se actualiza cada vez que se recibe un mensaje




In [ ]:
#para verificar si paramos seria
#check_lists(self.historial_origin)

In [76]:
check_lists(list_of_lists)

True

In [84]:
list_2.append(6)
list_2

[1, 2, 3, 4, 5, 6]

In [94]:
list1

[1, 2, 3, 4]

In [112]:
listado = [1,2,3]

In [110]:
numeroprueba = 4 #[1,0,1000,0]

In [113]:
(listado.copy() + [numeroprueba])[-3:]

[2, 3, 4]

In [87]:
#print the last 3 elements of list_2
listado[-3:]

[1, 2]